In [1]:
import openseespy.opensees as ops

In [2]:
"""I'm going to define a brick wall that looks like this:

_|___|___|___|___|___|
___|___|___|___|___|__
_|___|___|___|___|___|
___|___|___|___|___|__
_|___|___|___|___|___| Y
___|___|___|___|___|__
_|___|___|___|___|___|
___|___|___|___|___|__
_|___|___|___|___|___|
___|___|___|___|___|__
          X



It's going to use eight node brick elements for both the brick and the mortar, 
and a different material model for each, I'm assuming the wall is one brick thick in the Z direction.

 6 o-------7o
  /|       /|
5/ |     6/ |
o--------o  |
|  o-----|--o
| / 2    | / 3
|/       |/
o--------o
 1        4
"""

"I'm going to define a brick wall that looks like this:\n\n_|___|___|___|___|___|\n___|___|___|___|___|__\n_|___|___|___|___|___|\n___|___|___|___|___|__\n_|___|___|___|___|___| Y\n___|___|___|___|___|__\n_|___|___|___|___|___|\n___|___|___|___|___|__\n_|___|___|___|___|___|\n___|___|___|___|___|__\n          X\n\n\n\nIt's going to use eight node brick elements for both the brick and the mortar, \nand a different material model for each, I'm assuming the wall is one brick thick in the Z direction.\n\n 6 o-------7o\n  /|       /|\n5/ |     6/ |\no--------o  |\n|  o-----|--o\n| / 2    | / 3\n|/       |/\no--------o\n 1        4\n"

In [3]:
#Geometry Inputs

#Wall dimensions
noBricksX = 20
noBricksY = 20

#Brick Dimensions - meters
brickX = 0.215
brickY = 0.065
brickZ = 0.1025

#Mortar Dimensions
mortarThickness = 0.01

In [4]:
# #Material Inputs
# #fpc: concrete compressive strength at 28 days (compression is negative)
# #epsc0: concrete strain at maximum strength
# #fpcu: concrete crushing strength
# #epsU: concrete strain at crushing strength

# brickMaterialModel = 'concrete01'
# mortarMaterialModel = 'concrete01'

# brickMaterialParams = {'fpc':10, 'epsc0':10, 'fpcu':10, 'epsU':10}
# mortarMaterialParams = {'fpc':10, 'epsc0':10, 'fpcu':10, 'epsU':10}

In [5]:
import numpy as np

nNodesX = noBricksX*2
nNodesY = noBricksY*2
nNodesZ = 2

nodeArray = np.zeros(shape=(nNodesY, nNodesX, nNodesZ))

idx = 1
for i,j,k in np.ndindex(nNodesY, nNodesX, nNodesZ):
    nodeArray[i,j,k] = idx
    idx += 1


In [6]:
coordArray = np.zeros(shape=(nNodesY, nNodesX, 3))

X=np.zeros(shape=(nNodesX,))    
spacing = [brickX, mortarThickness]

for n in range(1, nNodesX):
    X[n] = X[n-1]+spacing[(np.mod(n-1,2))]
    

Y=np.zeros(shape=(nNodesY,))    
spacing = [brickY, mortarThickness]
for n in range(1, nNodesY):
    Y[n] = Y[n-1]+spacing[(np.mod(n-1,2))]
    
Z=np.zeros(shape=(nNodesZ,))    
spacing = [brickZ]

for n in range(1, nNodesZ):
    Z[n] = Z[n-1]+spacing[0]
    
wallX, wallY, wallZ = np.meshgrid(X,Y,Z)

wall = np.stack((wallX, wallY, wallZ),axis=3)

In [7]:
import opsvis as opsv

ops.wipe()
ops.model('Basic', '-ndm', 3, '-ndf', 3)

eleArray = []
matArray = []

for i,j,k in np.ndindex(nNodesY,nNodesX,nNodesZ):
    ops.node(int(nodeArray[i,j,k]), *wall[i,j,k])

for i,j,k in np.ndindex(nNodesY-1,nNodesX-1,nNodesZ-1):
    
    eleArray.append([nodeArray[i,j,k], nodeArray[i,j+1,k], nodeArray[i,j+1,k+1], nodeArray[i,j,k+1],
                    nodeArray[i+1,j,k], nodeArray[i+1,j+1,k], nodeArray[i+1,j+1,k+1], nodeArray[i+1,j,k+1]])
    if np.mod(i,2)==1 or np.mod(j,2)==1:
        mat='mortar'
    else:
        mat='brick'
    matArray.append(mat)
    
eleArray = np.array(eleArray).astype(int)

In [8]:
# ops.nDMaterial('DruckerPrager', matTag, K, G, sigmaY, rho, rhoBar, 
#             Kinf, Ko, delta1, delta2, H, theta, density, atmPressure=101e3)

#ops.nDMaterial('DruckerPrager', 1, 1E10, 1E10, 1E10, 1400.0, 1400.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0)
#ops.nDMaterial('DruckerPrager', 2, 1E5, 1E10, 1E10, 1400.0, 1400.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0)


#ops.nDMaterial('ElasticIsotropic', 1, 210E9, 0.3, 7800)
#ops.nDMaterial('ElasticIsotropic', 2, 210E9, 0.3, 7800)

ops.nDMaterial('Damage2p', 1, 9E6)
ops.nDMaterial('Damage2p', 2, 17E6)

eleTag = 1
brickMatTag = 1
mortarMatTag = 2

for m, eleNodes in enumerate(eleArray):
    eleNodes = [int(e)for e in eleNodes]
    if matArray[m] == 'brick':
        ops.element('stdBrick', eleTag, *eleNodes, brickMatTag)
        eleTag+= 1
        
    elif matArray[m] == 'mortar':
        ops.element('stdBrick', eleTag, *eleNodes, mortarMatTag)
        eleTag+= 1

WARNING material type Damage2p is unknown


OpenSeesError: See stderr output

In [ ]:
#fix all four edges
xMinNodes = nodeArray[:,0,:].flatten().astype(int)
xMaxNodes = nodeArray[:,-1,:].flatten().astype(int)

yMinNodes = nodeArray[0,1:-1,:].flatten().astype(int)
yMaxNodes = nodeArray[-1,1:-1,:].flatten().astype(int)

for support in np.concatenate([xMinNodes, xMaxNodes, yMinNodes, yMaxNodes]):
    ops.fix(int(support), *[1,1,1])
    
# for support in yMinNodes:
#     ops.fix(int(support), *[1,1,1])

In [ ]:
ops.timeSeries('Linear', 1)
ops.pattern('Plain', 1, 1)

for node in ops.getNodeTags():
    ops.load(node, *[0,0,1])

In [ ]:
import numpy as np
import os
import vfo.vfo as vfo

ControlNode=int(nodeArray[int(nodeArray.shape[0]/2), int(nodeArray.shape[0]/2), 0]) #somewhere near the moddle
ControlDOF=3
MaxDisp=1
DispIncr=0.1
NstepsPush=int(MaxDisp/DispIncr)

# Analysis options
ops.system('BandGeneral')
ops.numberer('Plain')
ops.constraints('Plain')
ops.test('NormDispIncr', 1.0e-5,  100 )
ops.integrator('DisplacementControl', ControlNode, 3, DispIncr)  # Control top node in Y direction with 0.01 increment
ops.algorithm('Newton')
ops.analysis('Static')

vfo.createODB("Masonry", "1")

dataPush = np.zeros((NstepsPush,*nodeArray.shape[:-1]))
for j in range(NstepsPush):
    ops.analyze(1)
    
    for k,l in np.ndindex(nodeArray.shape[:-1]):
        dataPush[j,k,l] = ops.nodeDisp(int(nodeArray[k,l,0]),3)


In [ ]:
fig, ax = plt.subplots(1,1,figsize=(10,10))
i=9
ax.contourf(dataPush[i])

In [ ]:
NstepsPush

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import vfo.vfo as vfo
import opsvis as opsv
%matplotlib notebook
fig, ax = plt.subplots(1,1,figsize=(10,10),subplot_kw=dict(projection='3d'))

#opsv.plot_model(node_labels=1,element_labels=0, ax=ax)
opsv.plot_defo(ax=ax, sfac=1)